In [26]:
import pandas as pd
from tqdm.auto import tqdm
import datetime

In [5]:


# Read a sample of the data
prefix = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
df_trip_data = pd.read_parquet(prefix + 'green_tripdata_2025-11.parquet')

taxi_zone_data = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv'
df_zone_data = pd.read_csv(taxi_zone_data)


(265, 4)

In [15]:
df_trip_data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [21]:
# November trips less than a mile
nov_trip_lt_1_mile = df[(df['lpep_pickup_datetime'] >= '2025-11-01') & (df['lpep_pickup_datetime'] < '2025-12-01') & (df['trip_distance'] <= 1)]

len(nov_trip_lt_1_mile)

8007

In [25]:
# Date of longest trip where trip distance is less than 100 miles
trips_lt_100_miles = df[(df['trip_distance'] < 100)]

longest_trip_id = trips_lt_100_miles['trip_distance'].idxmax()

row_longest_trip = trips_lt_100_miles.loc[longest_trip_id]

row_longest_trip['lpep_pickup_datetime']

Timestamp('2025-11-14 15:36:27')

In [45]:
# Pickup zone with largest total amount on Novemeber 18th

nov_18_trips = df[df['lpep_pickup_datetime'].dt.date == datetime.date(2025, 11, 18)]
nov_18_trips_with_zones = pd.merge(nov_18_trips, df_zone_data, left_on = 'PULocationID', right_on = 'LocationID', how= 'inner')


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,LocationID,Borough,Zone,service_zone
0,2,2025-11-18 08:07:00,2025-11-18 08:19:15,N,1.0,74,75,1.0,1.40,12.1,...,1.0,17.00,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone
1,2,2025-11-18 08:19:54,2025-11-18 08:29:08,N,1.0,75,236,1.0,1.33,10.0,...,1.0,17.10,1.0,1.0,2.75,0.0,75,Manhattan,East Harlem South,Boro Zone
2,2,2025-11-18 08:42:22,2025-11-18 08:53:49,N,1.0,74,236,1.0,1.97,12.1,...,1.0,19.62,1.0,1.0,2.75,0.0,74,Manhattan,East Harlem North,Boro Zone
3,2,2025-11-18 09:08:07,2025-11-18 09:14:44,N,1.0,74,166,1.0,0.86,7.9,...,1.0,11.00,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone
4,2,2025-11-18 09:37:57,2025-11-18 09:52:39,N,1.0,74,116,1.0,2.16,15.6,...,1.0,22.23,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone


In [42]:
nov_18_trip_zones_total_amt = nov_18_trips_with_zones.groupby('Zone', as_index=False)['total_amount'].sum()
zone_with_highest_total_amt_nov_18 = nov_18_trip_zones_total_amt[nov_18_trip_zones_total_amt['total_amount'] == nov_18_trip_zones_total_amt['total_amount'].max()]

,Zone,total_amount
39,East Harlem North,9281.92


In [55]:
# Novemeber trips and pickup Zone: East Harlem North - Drop off zone with max tip amount

nov_trips = df[(df['lpep_pickup_datetime'] >= '2025-11-01') & (df['lpep_pickup_datetime'] < '2025-12-01')]
nov_trips_with_zones = pd.merge(nov_trips, df_zone_data, left_on = 'PULocationID', right_on = 'LocationID', how= 'inner')
nov_trips_pickup_EHN = nov_trips_with_zones[nov_trips_with_zones['Zone'] == 'East Harlem North']

nov_trips_pickup_EHN_max_tip_id = nov_trips_pickup_EHN['tip_amount'].idxmax()

row_w_max_tip = nov_trips_pickup_EHN.loc[nov_trips_pickup_EHN_max_tip_id]

drop_zone_w__max_tip =  df_zone_data[ df_zone_data['LocationID'] == row_w_max_tip['DOLocationID'] ]

drop_off_zone_name = drop_zone_w__max_tip['Zone']

drop_off_zone_name

262    Yorkville West
Name: Zone, dtype: str